In [1]:
import numpy as np
import tensorflow as tf

# Create a Regression graph (DAG)

In [3]:
x_ph = tf.placeholder(tf.float32, name="Input_values")    
y_ph = tf.placeholder(tf.float32, name="Output_values")

In [4]:
m = tf.Variable(initial_value=0.1, name="m")
b = tf.Variable(initial_value=1.,  name="b")

In [5]:
# DAG: y = mx+b

y_hat = m*x_ph + b       # If you feed values, you will get o/p. But needs to be tuned for least loss possible

In [6]:
losses = tf.square(y_ph - y_hat, name="losses_from_every_batch_of_input")   # Element wise difference

In [7]:
cost = tf.sqrt( tf.reduce_sum(losses), name="final_cost_after_all_batches" )       # Cost on the current batch

In [8]:
# Training: Minimise the cost by changing m & b

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
set_to_optimize = optimizer.minimize(cost)       # a). Just single step descent,    b). using the current batch only
                                        # And in the due course only variables (not placeholder) will be updated

# Tensorboard visuals

In [10]:
# 1st create a summary operation that will dump all output as you instruct later
writer = tf.summary.FileWriter("./log_dir", graph=tf.get_default_graph())   # Dump the whole graph also

At this point the graph can be viewed. Open CMD and follow type below:<br>
1. cd < folder_path_of_the_log_dir_created_above >
2. activate < environment_name_in_which_tensorflow_installed >
3. tensorboard --logdir .
            Notice the dot at last
            Now you will get a URL link. Copy & paste it in browser. Violla, tensorboard is visible now


In [14]:
# Attach the summary-op to all nodes wherever you want to capture scalars. Summary-op was created above
loss_summ = tf.summary.histogram("losses", losses)
cost_summ = tf.summary.scalar('total_cost', cost)    
m_summ = tf.summary.scalar("slope", m)
b_summ = tf.summary.scalar("intercept", b)

In [9]:
init = tf.global_variables_initializer()

In [12]:
# You actual whole dataset (millions of points)
x = np.linspace(1,10,100000) + np.random.uniform(-1.5, 1.5, 100000)
y = np.linspace(1,10,100000) + np.random.uniform(-1.5, 1.5, 100000)

# Just keep on feeding your input one at a time, and fetch the result

In [15]:
# All setup by now
# Go ahead and feed in values and generate results

# But Graph will be traversed and executed by another object, named Session
sess = tf.Session()
sess.run(init)   # Needed for variable initialization. (as per the value you already told them)

tot_iterations = 500
batch_size = 100      # Mini batch

for i in range(tot_iterations):
    sample_idx = np.random.randint(0,x.size, size=batch_size)

    # Create one mini-batch
    x_sample = x[sample_idx]
    y_sample = y[sample_idx]

    # Run single step of optimization
    sess.run(set_to_optimize, feed_dict={x_ph:x_sample, y_ph:y_sample})
    
    #At the saem time generate loss , cost for tensorboard 
    loss_buffer = sess.run(loss_summ, feed_dict={x_ph:x_sample, y_ph:y_sample})
    cost_buffer = sess.run(cost_summ, feed_dict={x_ph:x_sample, y_ph:y_sample})
    m_buffer = sess.run(m_summ, feed_dict={x_ph:x_sample, y_ph:y_sample})
    b_buffer = sess.run(b_summ, feed_dict={x_ph:x_sample, y_ph:y_sample})
    # And dump into event file
    writer.add_summary(loss_buffer, i)
    writer.add_summary(cost_buffer, i)
    writer.add_summary(m_buffer, i)
    writer.add_summary(b_buffer, i)

#OK , you finished optimization on many mini-batches. Get the trained values of m & b
model_m, model_b = sess.run([m, b])
    
model_m, model_b

(0.8617424, 0.79837495)